![Shopping trolley in front of a laptop](./iStock-1249219777.jpg)

It's simple to buy any product with a click and have it delivered to your door. Online shopping has been rapidly evolving over the last few years, making our lives easier. But behind the scenes, e-commerce companies face a complex challenge that needs to be addressed. 

Uncertainty plays a big role in how the supply chains plan and organize their operations to ensure that the products are delivered on time. These uncertainties can lead to challenges such as stockouts, delayed deliveries, and increased operational costs.

You work for the Sales & Operations Planning (S&OP) team at a multinational e-commerce company. They need your help to assist in planning for the upcoming end-of-the-year sales. They want to use your insights to plan for promotional opportunities and manage their inventory. This effort is to ensure they have the right products in stock when needed and ensure their customers are satisfied with the prompt delivery to their doorstep.


## The Data

You are provided with a sales dataset to use. A summary and preview are provided below.

# Online Retail.csv

| Column     | Description              |
|------------|--------------------------|
| `'InvoiceNo'` | A 6-digit number uniquely assigned to each transaction |
| `'StockCode'` | A 5-digit number uniquely assigned to each distinct product |
| `'Description'` | The product name |
| `'Quantity'` | The quantity of each product (item) per transaction |
| `'UnitPrice'` | Product price per unit |
| `'CustomerID'` | A 5-digit number uniquely assigned to each customer |
| `'Country'` | The name of the country where each customer resides |
| `'InvoiceDate'` | The day and time when each transaction was generated `"MM/DD/YYYY"` |
| `'Year'` | The year when each transaction was generated |
| `'Month'` | The month when each transaction was generated |
| `'Week'` | The week when each transaction was generated (`1`-`52`) |
| `'Day'` | The day of the month when each transaction was generated (`1`-`31`) |
| `'DayOfWeek'` | The day of the weeke when each transaction was generated <br>(`0` = Monday, `6` = Sunday) |

In [1]:
# Import required libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import col, dayofmonth, month, year,  to_date, to_timestamp, weekofyear, dayofweek
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Initialize Spark session
my_spark = SparkSession.builder.appName("SalesForecast").getOrCreate()

# Importing sales data
sales_data = my_spark.read.csv(
    "Online Retail.csv", header=True, inferSchema=True, sep=",")

# Convert InvoiceDate to datetime 
sales_data = sales_data.withColumn("InvoiceDate", to_date(
    to_timestamp(col("InvoiceDate"), "d/M/yyyy H:mm")))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/15 06:46:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.sql import functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType # Added import for IntegerType

# Assuming sales_data is a Spark DataFrame
grouped_sales_data = sales_data.groupBy("Country", "StockCode", "InvoiceDate", "Year", "Month", "Day", "Week", "DayOfWeek")\
    .agg(F.sum("Quantity").alias("Quantity"), F.avg("UnitPrice").alias("avgPrice"))

#Creating train set
train_data = grouped_sales_data.filter(F.col("InvoiceDate") <= "2011-09-25")

#Creating test set
test_data = grouped_sales_data.filter(F.col("InvoiceDate") > "2011-09-25")

#Converting train and test sets to pandas dataframes
pd_daily_train_data = train_data.select("Country", "StockCode", "InvoiceDate", "Quantity").toPandas()
pd_daily_test_data = test_data.select("Country", "StockCode", "InvoiceDate", "Quantity").toPandas()

#Creating StringIndexer object for 'Country' column
country_indexer = StringIndexer(inputCol="Country", outputCol="CountryIndex").setHandleInvalid("keep")

#Creating StringIndexer object for 'StockCode' column
stock_code_indexer = StringIndexer(inputCol="StockCode", outputCol="StockCodeIndex").setHandleInvalid("keep")

#Creating VectorAssembler object
assembler = VectorAssembler(inputCols=["CountryIndex", "StockCodeIndex", "Year", "Month", "Day", "Week", "DayOfWeek", "avgPrice"], outputCol="features")

#Creating Random Forest object
rf = RandomForestRegressor(featuresCol="features", labelCol="Quantity", maxBins=4000)

#Creating pipeline
pipeline = Pipeline(stages=[country_indexer, stock_code_indexer, assembler, rf])

# Fit the pipeline to the training data
pipeline_model = pipeline.fit(train_data)

# Apply the model to the test data
test_predictions = pipeline_model.transform(test_data)

# Cast the prediction column to double
test_predictions = test_predictions.withColumn("prediction", F.col("prediction").cast("double"))

# Evaluate the model
mae_evaluator = RegressionEvaluator(labelCol="Quantity", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(test_predictions)
print("Mean Absolute Error: ", mae)

test_predictions = test_predictions.groupBy("Year","Week").agg(F.sum("prediction").alias("sum_prediction"))
test_predictions = test_predictions.filter((F.col("Week") == 39) & (F.col("Year") == 2011))
quantity_sold_w39 = test_predictions.select(F.col("sum_prediction").cast(IntegerType())).collect()[0][0] # Corrected col to F.col

print("Expected to be sold during the week 39 of 2011: ", quantity_sold_w39)

24/03/15 06:47:03 WARN DAGScheduler: Broadcasting large task binary with size 1007.4 KiB


24/03/15 06:47:05 WARN DAGScheduler: Broadcasting large task binary with size 1633.1 KiB


24/03/15 06:47:06 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB


Mean Absolute Error:  9.185509275434585


Expected to be sold during the week 39 of 2011:  87489
